    a. Problem Statetement

    b. Data Cleaning And Preprocessing

    c. Exploratory Data Analysis

    d. Modeling

    e. Conclusion

    f. Daftar Pustaka

Submisi Jupyter Notebook berisi kode dalam bahasa python serta 
interpretasi dan penjelasan hasil penemuan insight data dalam bentuk 
markdown.

### A. Problem Statement

<!-- - Apakah kenaikan kasus covid dapat dipicu oleh hari-hari besar?
- Apakah kasus covid untuk 2 bulan kedepannya akan mengalami kenaikan atau penurunan?
- Apakah ada pengaruh mobilitas pada kasus kenaikan covid kenaikan 


Rencana proyek
- Memberikan forecasting untuk bulan kedepannya agar masyarakat atau pemerintah bisa melakukan tindak preventif untuk mencegah kenaikan covid. -->

**Problem statement**

Pandemi covid-19 memberikan dampak yang besar terhadap laju mobilitas negara-negara, termasuk Indonesia. Hal itu disebabkan dengan kenaikan kasus covid yang terus menerus serta varian covid baru yang bermunculan. Mobilitas yang terhampat dapat mempengaruhi berbagai aspek, seperti : ekonomi, sosial, kesehatan, dan lain-lain. Oleh karena itu perlu adanya tindakan yang tepat untuk menanggulangi kasus covid 19.


**Tujuan**
* Memberikan hasil Analisis untuk mencari pola kenaikan kasus covid
* Menentukan apakah kasus covid dapat dipicu oleh hari-hari besar 
* Melakukan prediksi untuk beberapa bulan selanjutnya agar pemerintah atau masyarakat dapat memberikan langkah preventif

**Rencana proyek**
* Melakukan EDA
- Melakukan prediksi beberapa bulan kedepan menggunakan algoritma sarimax

### B. Data Cleaning And Preprocessing

In [206]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import numpy as np
plt.style.use('fivethirtyeight')

#### **a. Dataset Daily Update Data Agregat Covid-19 Jakarta**
pada dataset daily update terdapat 13 sheet, tetapi kami hanya menggunakan 5 sheet saja

##### Data Jakarta

In [5]:
df_DU_Jakarta = pd.read_excel('Daily Update Data Agregat Covid-19 Jakarta.xlsx', sheet_name='Data Jakarta')
df_DU_Jakarta.shape

(838, 14)

In [6]:
# check dataframe for missing values
df_DU_Jakarta.isnull().sum()

Tanggal              0
Jam                838
Tanggal Jam        838
Meninggal            0
Sembuh               0
Self Isolation       0
Masih Perawatan      0
Total Pasien         0
Positif Harian       0
Positif Aktif        0
Sembuh Harian        0
Tanpa Gejala       140
Bergejala          140
Belum Ada Data     140
dtype: int64

pada variabel jam dan tanggal jam memiliki keseluruhan data null sehingga dilakukan *dropping*, kemudian pada variabel tanpa gejala, bergejala, dan belum ada data tidak dilakukan *dropping* karena kami asumsikan akan menghilangkan informasi penting sehingga kami lakukan imputasi dengan mengisinya dengan nilai 0

In [7]:
df_DU_Jakarta.drop(['Jam','Tanggal Jam'],axis=1,inplace=True)
df_DU_Jakarta.fillna(0,inplace=True)

In [8]:
# check dataframe for information about data types
df_DU_Jakarta.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 838 entries, 0 to 837
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Tanggal          838 non-null    datetime64[ns]
 1   Meninggal        838 non-null    int64         
 2   Sembuh           838 non-null    int64         
 3   Self Isolation   838 non-null    int64         
 4   Masih Perawatan  838 non-null    int64         
 5   Total Pasien     838 non-null    int64         
 6   Positif Harian   838 non-null    int64         
 7   Positif Aktif    838 non-null    int64         
 8   Sembuh Harian    838 non-null    int64         
 9   Tanpa Gejala     838 non-null    float64       
 10  Bergejala        838 non-null    float64       
 11  Belum Ada Data   838 non-null    float64       
dtypes: datetime64[ns](1), float64(3), int64(8)
memory usage: 78.7 KB


Karena Tanggal sudah bertipe datetime sehingga tidak perlu dilakukan preprocessing

In [9]:
df_DU_Jakarta

,Tanggal,Meninggal,Sembuh,Self Isolation,Masih Perawatan,Total Pasien,Positif Harian,Positif Aktif,Sembuh Harian,Tanpa Gejala,Bergejala,Belum Ada Data
0,2020-03-01,0,0,0,0,0,0,0,0,0.0,0.0,0.0
1,2020-03-02,0,0,0,0,0,0,0,0,0.0,0.0,0.0
2,2020-03-03,1,0,0,2,3,3,2,0,0.0,0.0,0.0
3,2020-03-04,1,0,0,2,3,0,2,0,0.0,0.0,0.0
4,2020-03-05,3,0,0,4,7,4,4,0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
833,2022-06-12,15303,1236002,2000,214,1253519,322,2214,164,0.0,9.0,2205.0
834,2022-06-13,15304,1236184,2179,200,1253867,348,2379,182,0.0,14.0,2365.0
835,2022-06-14,15304,1236374,2446,260,1254384,517,2706,190,0.0,14.0,2692.0
836,2022-06-15,15304,1236528,2980,302,1255114,730,3282,154,0.0,15.0,3267.0


##### Daily Vaksinasi Wilayah

In [15]:
#vaksinasi wilayah
df_DU_DVW = pd.read_excel('Daily Update Data Agregat Covid-19 Jakarta.xlsx', sheet_name='Daily Vaksinasi Wilayah').iloc[:,:3].dropna(axis=0, how='any')
df_DU_DVW

,Tanggal,Cakupan Suntik 1,Cakupan Suntik 2
0,2021-06-12,19281.0,1173.0
1,2021-06-13,198.0,29.0
2,2021-06-14,79491.0,3691.0
3,2021-06-15,81545.0,3385.0
4,2021-06-16,92210.0,3436.0
...,...,...,...
364,2022-06-11,430.0,280.0
365,2022-06-12,300.0,194.0
366,2022-06-13,900.0,1272.0
367,2022-06-14,1028.0,1112.0


In [14]:
df_DU_DVW.isna().sum()

Tanggal             0
Cakupan Suntik 1    0
Cakupan Suntik 2    0
dtype: int64

In [13]:
df_DU_DVW.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 369 entries, 0 to 368
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Tanggal           369 non-null    datetime64[ns]
 1   Cakupan Suntik 1  369 non-null    float64       
 2   Cakupan Suntik 2  369 non-null    float64       
dtypes: datetime64[ns](1), float64(2)
memory usage: 11.5 KB


##### Vaksinasi Wilayah

In [29]:
#vaksinasi wilayah
df_DU_VW = pd.read_excel('Daily Update Data Agregat Covid-19 Jakarta.xlsx', sheet_name='Vaksinasi Wilayah',skiprows=1)
df_DU_VW.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44 entries, 0 to 43
Columns: 1118 entries, No. to Cakupan Suntik 2.370
dtypes: float64(4), int64(742), object(372)
memory usage: 384.4+ KB


In [44]:
df_DU_VW.isna().sum().sum()

0

In [30]:
df_DU_VW.head()

,No.,Kecamatan,Kota / Kabupaten,Target Total per Kecamatan,Target Harian per Kecamatan,Cakupan Suntik 1,Keterangan,Cakupan Suntik 2,Cakupan Suntik 1.1,Keterangan.1,...,Cakupan Suntik 2.367,Cakupan Suntik 1.368,Keterangan.368,Cakupan Suntik 2.368,Cakupan Suntik 1.369,Keterangan.369,Cakupan Suntik 2.369,Cakupan Suntik 1.370,Keterangan.370,Cakupan Suntik 2.370
0,1,Cakung,Jakarta Timur,515628,5333,74966,Tidak Tercapai,37590,897,Tidak Tercapai,...,41,35,Tidak Tercapai,46,56,Tidak Tercapai,48,NaN,NaN,NaN
1,2,Cengkareng,Jakarta Barat,519118,5041,85668,Tidak Tercapai,47581,429,Tidak Tercapai,...,48,29,Tidak Tercapai,54,33,Tidak Tercapai,35,NaN,NaN,NaN
2,3,Kali Deres,Jakarta Barat,406018,4295,42991,Tidak Tercapai,23222,162,Tidak Tercapai,...,25,44,Tidak Tercapai,29,25,Tidak Tercapai,28,NaN,NaN,NaN
3,4,Cilincing,Jakarta Utara,392136,4073,41494,Tidak Tercapai,22305,17,Tidak Tercapai,...,76,38,Tidak Tercapai,57,64,Tidak Tercapai,52,NaN,NaN,NaN
4,5,Duren Sawit,Jakarta Timur,404329,4006,41413,Tidak Tercapai,28993,7,Tidak Tercapai,...,31,26,Tidak Tercapai,27,32,Tidak Tercapai,25,NaN,NaN,NaN


In [31]:
df_DU_VW = df_DU_VW.drop(df_DU_VW.select_dtypes('float64'),axis=1)

In [32]:
df_DU_VW_grouped_kec = df_DU_VW.groupby(["No.","Kecamatan","Kota / Kabupaten","Target Total per Kecamatan","Target Harian per Kecamatan"]).sum().sum(axis=1).to_frame().rename(columns={0:'Total Vaksinasi'}).reset_index().drop(['No.','Target Total per Kecamatan','Target Harian per Kecamatan'],axis=1)
df_DU_VW_grouped_kec

,Kecamatan,Kota / Kabupaten,Total Vaksinasi
0,Cakung,Jakarta Timur,555303
1,Cengkareng,Jakarta Barat,694473
2,Kali Deres,Jakarta Barat,587065
3,Cilincing,Jakarta Utara,483607
4,Duren Sawit,Jakarta Timur,470542
5,Tanjung Priok,Jakarta Utara,782796
6,Jagakarsa,Jakarta Selatan,406610
7,Kebon Jeruk,Jakarta Barat,485189
8,Koja,Jakarta Utara,608483
9,Kebayoran Lama,Jakarta Selatan,461942


In [33]:
df_DU_VW_grouped_kota = df_DU_VW_grouped_kec.copy()
df_DU_VW_grouped_kota = df_DU_VW_grouped_kota.reset_index().drop(['Kecamatan','index'], axis=1).groupby(['Kota / Kabupaten']).sum()
df_DU_VW_grouped_kota

,Total Vaksinasi
Kota / Kabupaten,
Jakarta Barat,3889204
Jakarta Pusat,3905840
Jakarta Selatan,5564185
Jakarta Timur,4721845
Jakarta Utara,3348347
Kepulauan Seribu,45919


##### Vaksinasi Booster Wilayah

In [35]:
df_DU_VBW = pd.read_excel('Daily Update Data Agregat Covid-19 Jakarta.xlsx', sheet_name='Vaksinasi Booster Wilayah')
df_DU_VBW

,Tanggal,Kecamatan,Wilayah,Akumulasi Cakupan D3,Persentase D3 terhadap D2 (Masyarakat Rentan/Umum),Persentase D3 terhadap D2
0,2022-02-06,Cakung,Jakarta Timur,12745,0.0672,0.0460
1,2022-02-06,Cengkareng,Jakarta Barat,16789,0.0601,0.0513
2,2022-02-06,Kali Deres,Jakarta Barat,12736,0.0516,0.0442
3,2022-02-06,Cilincing,Jakarta Utara,5756,0.0272,0.0243
4,2022-02-06,Duren Sawit,Jakarta Timur,20377,0.1087,0.0853
...,...,...,...,...,...,...
5715,2022-06-15,Taman Sari,Jakarta Barat,61254,0.5997,0.5287
5716,2022-06-15,Tambora,Jakarta Barat,57592,0.3853,0.3315
5717,2022-06-15,Tanah Abang,Jakarta Pusat,71266,0.2584,0.2035
5718,2022-06-15,Tanjung Priok,Jakarta Utara,130737,0.3808,0.3286


In [40]:
df_DU_VBW.isna().sum()

Tanggal                                               0
Kecamatan                                             0
Wilayah                                               0
Akumulasi Cakupan D3                                  0
Persentase D3 terhadap D2 (Masyarakat Rentan/Umum)    0
Persentase D3 terhadap D2                             0
dtype: int64

In [41]:
df_DU_VBW.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5720 entries, 0 to 5719
Data columns (total 6 columns):
 #   Column                                              Non-Null Count  Dtype         
---  ------                                              --------------  -----         
 0   Tanggal                                             5720 non-null   datetime64[ns]
 1   Kecamatan                                           5720 non-null   object        
 2   Wilayah                                             5720 non-null   object        
 3   Akumulasi Cakupan D3                                5720 non-null   int64         
 4   Persentase D3 terhadap D2 (Masyarakat Rentan/Umum)  5720 non-null   float64       
 5   Persentase D3 terhadap D2                           5720 non-null   float64       
dtypes: datetime64[ns](1), float64(2), int64(1), object(2)
memory usage: 268.2+ KB


In [39]:
df_DU_VBW_grouped = df_DU_VBW.groupby(['Tanggal','Wilayah']).agg({'Akumulasi Cakupan D3':'sum','Persentase D3 terhadap D2 (Masyarakat Rentan/Umum)':'mean','Persentase D3 terhadap D2':'mean'}).reset_index()
df_DU_VBW_grouped

,Tanggal,Wilayah,Akumulasi Cakupan D3,Persentase D3 terhadap D2 (Masyarakat Rentan/Umum),Persentase D3 terhadap D2
0,2022-02-06,Jakarta Barat,124888,0.086900,0.068612
1,2022-02-06,Jakarta Pusat,184916,0.151487,0.107938
2,2022-02-06,Jakarta Selatan,227642,0.114440,0.086190
3,2022-02-06,Jakarta Timur,184524,0.105720,0.083080
4,2022-02-06,Jakarta Utara,91900,0.072667,0.055100
...,...,...,...,...,...
775,2022-06-15,Jakarta Pusat,689767,0.469450,0.398838
776,2022-06-15,Jakarta Selatan,976313,0.389460,0.329740
777,2022-06-15,Jakarta Timur,944297,0.465350,0.392510
778,2022-06-15,Jakarta Utara,579402,0.410350,0.338600


##### Data Input Daily Vaksinasi Anak

In [47]:
df_DU_Vaks_Anak = pd.read_excel('Daily Update Data Agregat Covid-19 Jakarta.xlsx', sheet_name='Data Input Daily Vaksinasi Anak')
df_DU_Vaks_Anak

,Tanggal,Dosis,JAKARTA PUSAT,JAKARTA UTARA,JAKARTA BARAT,JAKARTA SELATAN,JAKARTA TIMUR,ADM. KEPULAUAN SERIBU,Total
0,2021-12-14,1,1860,1447,1738,2099,2440,105,9689
1,2021-12-15,1,4366,6742,6466,6810,7022,451,31857
2,2021-12-16,1,6981,9442,5807,6638,9279,595,38742
3,2021-12-17,1,6171,9621,7251,6009,11995,393,41440
4,2021-12-18,1,821,400,106,536,760,10,2633
...,...,...,...,...,...,...,...,...,...
345,2022-06-13,2,11,34,17,27,34,0,123
346,2022-06-14,1,13,29,21,21,32,0,116
347,2022-06-14,2,20,53,36,42,54,0,205
348,2022-06-15,1,13,21,19,29,43,0,125


In [48]:
df_DU_Vaks_Anak.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 350 entries, 0 to 349
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   Tanggal                350 non-null    datetime64[ns]
 1   Dosis                  350 non-null    int64         
 2   JAKARTA PUSAT          350 non-null    int64         
 3   JAKARTA UTARA          350 non-null    int64         
 4   JAKARTA BARAT          350 non-null    int64         
 5   JAKARTA SELATAN        350 non-null    int64         
 6   JAKARTA TIMUR          350 non-null    int64         
 7   ADM. KEPULAUAN SERIBU  350 non-null    int64         
 8   Total                  350 non-null    int64         
dtypes: datetime64[ns](1), int64(8)
memory usage: 24.7 KB


In [49]:
df_DU_Vaks_Anak.isna().sum()

Tanggal                  0
Dosis                    0
JAKARTA PUSAT            0
JAKARTA UTARA            0
JAKARTA BARAT            0
JAKARTA SELATAN          0
JAKARTA TIMUR            0
ADM. KEPULAUAN SERIBU    0
Total                    0
dtype: int64

#### **b. Dataset Mobilitas**

In [62]:
df_M = pd.read_csv('2020_ID_Region_Mobility_Report.csv')
df_M.shape

(11235, 15)

In [63]:
df_M.isna().sum()

country_region_code                                       0
country_region                                            0
sub_region_1                                            321
sub_region_2                                          11235
metro_area                                            11235
iso_3166_2_code                                         321
census_fips_code                                      11235
place_id                                                  0
date                                                      0
retail_and_recreation_percent_change_from_baseline       50
grocery_and_pharmacy_percent_change_from_baseline        99
parks_percent_change_from_baseline                      118
transit_stations_percent_change_from_baseline           391
workplaces_percent_change_from_baseline                   0
residential_percent_change_from_baseline                  0
dtype: int64

In [64]:
df_M.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11235 entries, 0 to 11234
Data columns (total 15 columns):
 #   Column                                              Non-Null Count  Dtype  
---  ------                                              --------------  -----  
 0   country_region_code                                 11235 non-null  object 
 1   country_region                                      11235 non-null  object 
 2   sub_region_1                                        10914 non-null  object 
 3   sub_region_2                                        0 non-null      float64
 4   metro_area                                          0 non-null      float64
 5   iso_3166_2_code                                     10914 non-null  object 
 6   census_fips_code                                    0 non-null      float64
 7   place_id                                            11235 non-null  object 
 8   date                                                11235 non-null  object 


In [65]:
df_M.drop(['sub_region_2','metro_area','census_fips_code','place_id'],axis=1,inplace=True)

In [66]:
df_M

,country_region_code,country_region,sub_region_1,iso_3166_2_code,date,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline
0,ID,Indonesia,NaN,NaN,2020-02-15,-2.0,-2.0,-8.0,1.0,5,1
1,ID,Indonesia,NaN,NaN,2020-02-16,-3.0,-3.0,-7.0,-4.0,2,2
2,ID,Indonesia,NaN,NaN,2020-02-17,-3.0,-4.0,-7.0,-1.0,3,1
3,ID,Indonesia,NaN,NaN,2020-02-18,-3.0,-2.0,-4.0,1.0,2,1
4,ID,Indonesia,NaN,NaN,2020-02-19,-3.0,-4.0,-3.0,0.0,1,1
...,...,...,...,...,...,...,...,...,...,...,...
11230,ID,Indonesia,West Sumatra,ID-SB,2020-12-27,4.0,28.0,29.0,-8.0,-2,4
11231,ID,Indonesia,West Sumatra,ID-SB,2020-12-28,2.0,25.0,32.0,-3.0,-30,9
11232,ID,Indonesia,West Sumatra,ID-SB,2020-12-29,1.0,26.0,32.0,2.0,-30,8
11233,ID,Indonesia,West Sumatra,ID-SB,2020-12-30,11.0,38.0,53.0,5.0,-28,8


### C. Exploratory Data Analysis

#### **a. Dataset Daily Update Data Agregat Covid-19 Jakarta**

##### Data Jakarta

In [68]:
df_DU_Jakarta

,Tanggal,Meninggal,Sembuh,Self Isolation,Masih Perawatan,Total Pasien,Positif Harian,Positif Aktif,Sembuh Harian,Tanpa Gejala,Bergejala,Belum Ada Data
0,2020-03-01,0,0,0,0,0,0,0,0,0.0,0.0,0.0
1,2020-03-02,0,0,0,0,0,0,0,0,0.0,0.0,0.0
2,2020-03-03,1,0,0,2,3,3,2,0,0.0,0.0,0.0
3,2020-03-04,1,0,0,2,3,0,2,0,0.0,0.0,0.0
4,2020-03-05,3,0,0,4,7,4,4,0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
833,2022-06-12,15303,1236002,2000,214,1253519,322,2214,164,0.0,9.0,2205.0
834,2022-06-13,15304,1236184,2179,200,1253867,348,2379,182,0.0,14.0,2365.0
835,2022-06-14,15304,1236374,2446,260,1254384,517,2706,190,0.0,14.0,2692.0
836,2022-06-15,15304,1236528,2980,302,1255114,730,3282,154,0.0,15.0,3267.0


In [84]:
fig = px.line(x='Tanggal', y=['Total Pasien','Meninggal','Sembuh'], data_frame=df_DU_Jakarta,title='Total Pasien Covid, Pasien Meninggal, dan Pasien Sembuh pada Maret 2020 hinga Juni 2022')
fig.add_annotation(x='2021-06-20', y=680000,
            text="Terjadi Lonjakan covid",
            align="center",
            arrowhead=2,
            arrowsize=1,
            arrowwidth=2,
            # yshift=40,
            # xshift=10,
            ax=-50,
            ay=-30,
            bordercolor="#333",
            borderwidth=2,
            borderpad=4,
            bgcolor="#F15412",
            opacity=0.8,
            font_color="#fff")

fig.add_annotation(x='2022-02-01', y=1100000,
            text="Terjadi Lonjakan covid",
            align="center",
            arrowhead=2,
            arrowsize=1,
            arrowwidth=2,
            # yshift=40,
            # xshift=10,
            ax=-50,
            ay=-30,
            bordercolor="#333",
            borderwidth=2,
            borderpad=4,
            bgcolor="#F15412",
            opacity=0.8,
            font_color="#fff")
fig.show()

Interpretasi

Pada diagram garis diatas menampilkan berkembangan jumlah pasien yang terkena covid, pasien yang meninggal, dan pasien yang sembuh dari tahun 2020 bulan maret hingga tahun 2022 bulan Juni.
Lalu dapat dilihat bahwa terjadi 2 kenaikan yang signifikan. Kenaikan pertama pada tahun 2021 bulan Juni akhir hingga bulan juli akhir  dan kenaikan kedua pada awal febuari 2022 hingga akhir febuari.

Asumsi kami pada kenaikan pertama hal tersebut dikarenakan pada pada tanggal 20 Juli 2021 terdapat idul adha sehingga terjadi arus mudik dan arus balik. Lalu terdapat varian baru seperti Alpha, Beta, dan Delta. Hal ini juga didukung oleh berita [suara.com](https://www.suara.com/news/2021/06/27/003810/covid-19-di-indonesia-menggila-ini-data-lonjakan-kasus-sepanjang-bulan-juni). Kemudian untuk kenaikan kedua pada awal febuari terjadi karena gelombang kedua covid untuk varian omicron berdasarkan berita [nasional.kontan.co.id](https://nasional.kontan.co.id/news/kasus-covid-19-24-februari-2022-tambah-57426-tapi-ada-kabar-baik)

In [201]:
px.line(x='Tanggal', y=['Positif Harian'], data_frame=df_DU_Jakarta,title='Data Pasien covid Positif Harian',color_discrete_sequence=['red'])

Interpretasi

Hal tersebut dapat dilihat lebih jelas pada data covid positif harian yang memiliki kenaikan pada bulan juli dan febuari.

In [100]:
print('Data positif covid tanpa gejala         : ',df_DU_Jakarta['Tanpa Gejala'].sum())
print('Data positif covid dengan gejala        : ',df_DU_Jakarta['Bergejala'].sum())
print('Data positif covid yang tidak diketahui : ',df_DU_Jakarta['Belum Ada Data'].sum())

Data positif covid tanpa gejala         :  2584819.0
Data positif covid dengan gejala        :  3501548.0
Data positif covid yang tidak diketahui :  5389258.0


In [127]:
df_DU_Jakarta[['Tanpa Gejala','Bergejala','Belum Ada Data']].sum().sum()

11475625.0

In [184]:
fig = px.pie(names=['Tanpa Gejala','Bergejala','Belum Ada Data'],values=df_DU_Jakarta[['Tanpa Gejala','Bergejala','Belum Ada Data']].sum().values,title='Data Pasien covid Positif Harian', color_discrete_sequence=px.colors.sequential.RdBu,hole=.4)
fig.update_traces(textposition='inside', textinfo='percent+label', marker=dict(line=dict(color='#000000', width=2)))
fig.update_layout(
    # Add annotations in the center of the donut pies.
    annotations=[dict(text='Total :', x=0.5, y=0.56, font_size=15, showarrow=False),
                 dict(text='11475625.0', x=0.5, y=0.46, font_size=13, showarrow=False)])
fig.show()

Interpretasi

Dari 11475625 pasien covid 30.5% mengalami gejala dan 22.5% pasien covid tidak mengalami gejala.

dari perbedaan yang sedikit tersebut yaitu 8%, kami asumsikan gejala tidak dapat menjadi faktor utama untuk mengetahui bahwa apakah seorang itu mengalami covid atau tidak.

In [187]:
df_DU_Jakarta[['Sembuh','Meninggal']].sum()

Sembuh       407740901
Meninggal      6277939
dtype: int64

In [194]:
df_DU_Jakarta[['Sembuh','Meninggal']].sum().sum()

414018840

In [200]:
fig = px.pie(names=['Sembuh','Meninggal'],values=[407740901,6277930],title='Persentase Kesembuhan Covid',color_discrete_sequence=px.colors.sequential.RdBu,hole=.4)
fig.update_traces(textinfo='percent+label', marker=dict(line=dict(color='#000000', width=2)))
fig.update_layout(
    # Add annotations in the center of the donut pies.
    annotations=[dict(text='Total :', x=0.5, y=0.56, font_size=15, showarrow=False),
                 dict(text='414018840', x=0.5, y=0.46, font_size=13, showarrow=False)])
fig.show()

Interpretasi

Dari 414018840 Pasien 98.48% pasien covid sembuh dan 1.52% pasien meninggal dengan angka kesembuhan pasien yang tinggi kami asumsikan bahwa covid bukanlah penyakit yang mematikan, tetapi memiliki penularan yang tinggi oleh karena itu mesti tetap waspada

##### Daily Vaksinasi Wilayah

In [202]:
df_DU_DVW

,Tanggal,Cakupan Suntik 1,Cakupan Suntik 2
0,2021-06-12,19281.0,1173.0
1,2021-06-13,198.0,29.0
2,2021-06-14,79491.0,3691.0
3,2021-06-15,81545.0,3385.0
4,2021-06-16,92210.0,3436.0
...,...,...,...
364,2022-06-11,430.0,280.0
365,2022-06-12,300.0,194.0
366,2022-06-13,900.0,1272.0
367,2022-06-14,1028.0,1112.0


In [203]:
px.line(x='Tanggal',y=['Cakupan Suntik 1','Cakupan Suntik 2'],data_frame=df_DU_DVW,title='Cakupan Suntik Covid-19 Jakarta')

In [204]:
df_DU_DVW[['Cakupan Suntik 1','Cakupan Suntik 2']].sum()

Cakupan Suntik 1    8594205.000
Cakupan Suntik 2    8207295.023
dtype: float64

In [240]:
df_DU_DVW[['Cakupan Suntik 1','Cakupan Suntik 2']].sum().sum()

16801500.023000002

In [255]:
# import plotly.graph_objs as go

# data = [
#     # Portfolio (inner donut)
#     go.Pie(values=[8594205,8207295,261950861],
#     labels=['Reds','Blues','Greens'],
#     domain={'x':[0.2,0.8], 'y':[0.1,0.9]},
#     hole=0.5,
#     direction='clockwise',
#     sort=False,
#     marker={'colors':['#CB4335','#2E86C1']}),
    
#     # Individual components (outer donut)
#     go.Pie(values=[16801500,278752361],
#     labels=['Medium Red','Light Red','Medium Blue','Light Blue'],
#     domain={'x':[0.1,0.9], 'y':[0,1]},
#     hole=0.75,
#     direction='clockwise',
#     sort=False,
#     marker={'colors':['#EC7063','#85C1E9','#F1948A','#5DADE2']},
#     showlegend=False)
# ]

# fig = go.Figure(data=data, layout={'title':'Nested Pie Chart'})
# fig.show()

In [260]:
fig = px.pie(names=['Cakupan Suntik 1 ','Cakupan Suntik 2 ','Total Penduduk Indonesia'],values=[8594205,8207295,278752361],title='Persentase Kesembuhan Covid',color_discrete_sequence=px.colors.sequential.RdBu,hole=.4)
fig.update_traces(textinfo='percent+label', marker=dict(line=dict(color='#000000', width=2)))
fig.update_layout(
    # Add annotations in the center of the donut pies.
    annotations=[dict(text='Total :', x=0.5, y=0.56, font_size=15, showarrow=False),
                 dict(text='278.752.361', x=0.5, y=0.46, font_size=13, showarrow=False)])
fig.show()

Interpretasi
Berdasarkan https://nasional.kompas.com/read/2022/04/27/03000051/jumlah-penduduk-indonesia-2022?page=all#:~:text=Worldometer%20merilis%20data%20jumlah%20penduduk,Perserikatan%20Bangsa%2Dbangsa%20atau%20PBB.&text=Jumlah%20penduduk%20Indonesia%20merupakan%203%2C51%20persen%20dari%20total%20penduduk%20dunia. jumlah penduduk di indonesia adalah 278 juta dan baru 2.91% cakupan suntik 1 dan 2.78% cakupan suntik 2

##### Vaksinisasi WIlayah

In [264]:
df_DU_VW

,No.,Kecamatan,Kota / Kabupaten,Target Total per Kecamatan,Target Harian per Kecamatan,Cakupan Suntik 1,Keterangan,Cakupan Suntik 2,Cakupan Suntik 1.1,Keterangan.1,...,Cakupan Suntik 2.366,Cakupan Suntik 1.367,Keterangan.367,Cakupan Suntik 2.367,Cakupan Suntik 1.368,Keterangan.368,Cakupan Suntik 2.368,Cakupan Suntik 1.369,Keterangan.369,Cakupan Suntik 2.369
0,1,Cakung,Jakarta Timur,515628,5333,74966,Tidak Tercapai,37590,897,Tidak Tercapai,...,2,56,Tidak Tercapai,41,35,Tidak Tercapai,46,56,Tidak Tercapai,48
1,2,Cengkareng,Jakarta Barat,519118,5041,85668,Tidak Tercapai,47581,429,Tidak Tercapai,...,1,53,Tidak Tercapai,48,29,Tidak Tercapai,54,33,Tidak Tercapai,35
2,3,Kali Deres,Jakarta Barat,406018,4295,42991,Tidak Tercapai,23222,162,Tidak Tercapai,...,0,32,Tidak Tercapai,25,44,Tidak Tercapai,29,25,Tidak Tercapai,28
3,4,Cilincing,Jakarta Utara,392136,4073,41494,Tidak Tercapai,22305,17,Tidak Tercapai,...,3,52,Tidak Tercapai,76,38,Tidak Tercapai,57,64,Tidak Tercapai,52
4,5,Duren Sawit,Jakarta Timur,404329,4006,41413,Tidak Tercapai,28993,7,Tidak Tercapai,...,3,15,Tidak Tercapai,31,26,Tidak Tercapai,27,32,Tidak Tercapai,25
5,6,Tanjung Priok,Jakarta Utara,377764,3728,81521,Tidak Tercapai,51856,1031,Tidak Tercapai,...,3,51,Tidak Tercapai,67,85,Tidak Tercapai,63,69,Tidak Tercapai,36
6,7,Jagakarsa,Jakarta Selatan,329378,3291,33493,Tidak Tercapai,21193,39,Tidak Tercapai,...,0,27,Tidak Tercapai,14,21,Tidak Tercapai,24,11,Tidak Tercapai,17
7,8,Kebon Jeruk,Jakarta Barat,327860,3285,53785,Tidak Tercapai,38086,366,Tidak Tercapai,...,2,20,Tidak Tercapai,26,11,Tidak Tercapai,30,23,Tidak Tercapai,24
8,9,Koja,Jakarta Utara,309976,3253,40095,Tidak Tercapai,23217,188,Tidak Tercapai,...,1,21,Tidak Tercapai,38,26,Tidak Tercapai,24,30,Tidak Tercapai,17
9,10,Kebayoran Lama,Jakarta Selatan,299047,3018,50278,Tidak Tercapai,39551,150,Tidak Tercapai,...,4,11,Tidak Tercapai,37,20,Tidak Tercapai,21,27,Tidak Tercapai,31


In [265]:
df_DU_VW_grouped_kota

,Total Vaksinasi
Kota / Kabupaten,
Jakarta Barat,3889204
Jakarta Pusat,3905840
Jakarta Selatan,5564185
Jakarta Timur,4721845
Jakarta Utara,3348347
Kepulauan Seribu,45919


In [262]:
df_DU_VW_grouped_kota.sum()

Total Vaksinasi    21475340
dtype: int64

In [268]:
px.bar(df_DU_VW_grouped_kota,color=df_DU_VW_grouped_kota.index,text_auto=True)

### F. Daftar Pustaka

https://www.suara.com/news/2021/06/27/003810/covid-19-di-indonesia-menggila-ini-data-lonjakan-kasus-sepanjang-bulan-juni